In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random

## **1. Data Definition**

### Customers Data

In [ ]:
num_customers = 11
customer_demands = [0, 5, 20, 10, 20, 85, 65, 30, 20, 70, 30]

### Vehicles Data

In [ ]:
# vehicles
num_vehicles = 6
vehicle_capacity = 100

### Routes Costs

In [ ]:
# Routes costs
distance_matrix = [
    [0, 13, 6, 55, 93, 164, 166, 168, 169, 241, 212],
    [13, 0, 11, 66, 261, 175, 177, 179, 180, 239, 208],
    [6, 11, 0, 60, 97, 168, 171, 173, 174, 239, 209],
    [55, 66, 60, 0, 82, 113, 115, 117, 117, 295, 265],
    [93, 261, 97, 82, 0, 113, 115, 117, 118, 333, 302],
    [164, 175, 168, 113, 113, 0, 6, 7, 2, 403, 374],
    [166, 177, 171, 115, 115, 6, 0, 8, 7, 406, 376],
    [168, 179, 173, 117, 117, 4, 8, 0, 3, 408, 378 ],
    [169, 180, 174, 117, 118, 3, 7, 3, 0, 409, 379],
    [241, 239, 239, 295, 333, 403, 406, 408, 409, 0, 46],
    [212, 208, 209, 265, 302, 374, 376, 378, 379, 46, 0]
]

distances = pd.DataFrame(distance_matrix, index=range(1, 12), columns=range(1, 12))
distances

## **2. Graph**

In [ ]:
# initialize the graph
G = nx.Graph()

# add nodes
for i in range(1, 12):
    G.add_node(i)

# add edges
for i in range(1, 12):
    for j in range(i + 1, 12):
        if distance_matrix[i-1][j-1] != 0:
            G.add_edge(i, j, weight=distance_matrix[i-1][j-1])

# draw the graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=10, font_weight='bold')
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=6)
plt.title('Customer Graph with Distances')
plt.show()

## **3. Genetic Algorithm**

In [ ]:
# Genetic Algotrithm class
class GeneticAlgorithm:
    def __init__(self, num_customers, vehicle_capacity, distance_matrix, customer_demands):
        self.num_customers = num_customers
        self.vehicle_capacity = vehicle_capacity
        self.distance_matrix = distance_matrix
        self.customer_demands = customer_demands

    def initialize_population(self):
        population = []
        remaining_customers = list(range(1, self.num_customers + 1))  # List of remaining customers
        random.shuffle(remaining_customers)  # Shuffle the customer list to randomize assignment
        while remaining_customers:
            route = [0]  # Initialize route with depot
            load = 0
            while remaining_customers:
                next_customer = remaining_customers[-1]
                if load + self.customer_demands[next_customer - 1] <= self.vehicle_capacity:
                    route.append(next_customer)
                    load += self.customer_demands[next_customer - 1]
                    remaining_customers.pop()
                else:
                    break
            route.append(0)  # Add depot at the end
            population.append(route)
        return population

    def evaluate_route(self, route):
        cost = 0
        load = 0
        for i in range(len(route) - 1):
            from_node, to_node = route[i], route[i+1]
            if to_node == 0 or to_node > self.num_customers:
                continue
            cost += self.distance_matrix[from_node - 1][to_node - 1]  # Adjust indexing
            load += self.customer_demands[to_node - 1]  # Adjust indexing
            if load > self.vehicle_capacity:
                cost += 1000
                load = 0
        return cost

    def mutate_route(self, route):
        mutated_route = route[:]
        idx_range = range(1, len(mutated_route) - 1)
        if len(idx_range) < 2:
            return mutated_route  # No mutation possible
        idx1, idx2 = random.sample(idx_range, 2)  # Exclude depot indices
        
        # Ensure that idx2 is greater than idx1
        if idx1 > idx2:
            idx1, idx2 = idx2, idx1
        
        # Reverse the sub-route between idx1 and idx2
        mutated_route = route[:idx1] + list(reversed(route[idx1:idx2+1])) + route[idx2+1:]
        
        # Check capacity constraint
        current_demand = sum(self.customer_demands[customer - 1] for customer in mutated_route[1:-1])
        if current_demand > self.vehicle_capacity:
            return route  # Revert mutation
        return mutated_route

    def genetic_algorithm(self, population, generations):
        for gen in range(generations):
            new_population = []
            for routes in population:
                new_routes = []
                for route in routes:
                    new_route = self.mutate_route(route)
                    new_routes.append(new_route)
                new_population.append(new_routes)
            
            population = new_population

            # Evaluate population
            all_routes = [route for routes in population for route in routes]
            all_costs = [self.evaluate_route(route) for route in all_routes]

            # Find the best route for each vehicle
            best_routes = []
            best_costs = []
            for i in range(num_vehicles):
                routes_for_vehicle = all_routes[i::num_vehicles]
                costs_for_vehicle = all_costs[i::num_vehicles]
                if costs_for_vehicle:
                    best_route_idx = np.argmin(costs_for_vehicle)
                    best_routes.append(routes_for_vehicle[best_route_idx])
                    best_costs.append(costs_for_vehicle[best_route_idx])
                else:
                    best_routes.append([])
                    best_costs.append(0)

        return best_routes, best_costs
        

## **4. Genetic Algorithm Execution**

In [ ]:
# main
if __name__ == "__main__":
    
    # an instance of the genetic algorithm class
    ga = GeneticAlgorithm(num_customers, vehicle_capacity, distance_matrix, customer_demands)
    
    # initialize population
    population = ga.initialize_population()
    best_routes, best_costs = ga.genetic_algorithm([population], generations=100)

    # print best routes
    for i in range(len(best_routes)):
        print(f"Vehicle {i+1}: Best Cost = {best_costs[i]}, Best Route = {best_routes[i]}")

    # print total cost
    print(f'Total Cost is: {sum(best_costs)}')

## **5. Optimal Routes Visualization**

In [ ]:
import matplotlib.pyplot as plt

def visualize_routes(best_routes, distance_matrix):
    # Plotting the nodes
    plt.figure(figsize=(10, 6))
    for i in range(len(distance_matrix)):
        plt.plot(distance_matrix[i][0], distance_matrix[i][1], 'o', markersize=10, label=f'Customer {i+1}')

    # Plotting the routes
    for i, route in enumerate(best_routes):
        if route:
            route_x = [distance_matrix[node - 1][0] for node in route]  # Adjust indexing
            route_y = [distance_matrix[node - 1][1] for node in route]  # Adjust indexing
            plt.plot(route_x, route_y, linestyle='-', linewidth=2, label=f'Vehicle {i+1} Route')
        else:
            pass

    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Visualization of Best Routes')
    plt.legend()
    plt.grid(True)
    plt.show()

# Assuming distance_matrix is a list of (x, y) coordinates for each node
# For example: distance_matrix = [(x1, y1), (x2, y2), ...]
visualize_routes(best_routes, distance_matrix)


## **6. Results Analysis**

The solution of this $NP-Hard$ problem with $Genetic Algorithms$ is not necessarily the optimal one.
elaborate...